# <p style="text-align: center;"> Sudoku et Reinforcement Learning </p>

#### <div style="text-align: right"> Auteur : Hugues Gallier </div>

J'ai souhaité faire ce projet sur un jeu bien connu, aux règles très simples : le jeu du sudoku. Celui-ci remonte à la fin du XIXème siècle en France [[1]](#bibliography) mais a continué de poser de nombreux problèmes mathématiques [[2]](#bibliography) jusqu'à très récemment.
En effet, ce n'est que dans les années 2010 [[3]](#bibliography) qu'une équipe de chercheurs prouve la conjecture selon laquelle toute grille de 16 indices ou moins ne possède pas d'unique solution ; cette preuve leur aura nécessité l'équivalent de 7 millions d'heures de CPU afin d'envisager tous les cas possibles.

Ce jeu reste aujourd'hui très intéressant afin de tester des approches de Machine Learning ou de Reinforcement Learning. En effet, même si les algorithmes déterministes marchent très bien (l'algorithme Backtrack notamment, pouvant bénéficier d'une optimisation très efficace, voir Dancing Links de Knuth [[4]](#bibliography)), la taille des grilles finales possibles reste très conséquente (environ 6e21, voir de nouveau [[2]](#bibliography)), et ce problème peut être considéré comme un exemple d'application intéressante pour des méthodes de type Reinforcement Learning.

Ainsi, je comparerai dans ce notebook plusieurs méthodes en termes de nombre de grilles différentes considérées avant d'aboutir à la solution (ce que j'appelerai **itération** par la suite est le fait de changer de grille considérée pour une nouvelle grille). En effet, le temps d'exécution des différents algorithmes repose sur l'optimisation de ceux-ci et sur les CPU ou GPU utilisées ; le nombre d'itérations, lui, ne repose que sur les méthodes que l'on considère.

Ce notebook sera divisé en plusieurs parties:

- [Partie I](#partie_1): Algorithme Backtrack
- [Partie II](#partie_2): Approche Deep Learning
- [Partie III](#partie_3): Monte Carlo Tree Search
- [Partie IV](#partie_4): AlphaSudoku: implémentation simplifiée et adaptée au Sudoku d'AlphaGo
- [Partie V](#partie_5): Comparaison de ces algorithmes sur 1000 nouvelles grilles

## Introduction : les données utilisées

J'utiliserai tout au long de ce notebook les données présentes [ici](https://www.kaggle.com/radcliffe/3-million-sudoku-puzzles-with-ratings), qui contiennent 3 millions de grilles avec leurs solutions respectives. Ce dataset est intéressant car il contient des grilles difficiles à résoudre, ce qui n'est pas le cas de tous les datasets que j'ai pu trouver. J'ai entraîné le réseau de neurones utilisé dans II et IV sur les 2 premiers millions de lignes, et testé tous les algorithmes sur 700 grilles issues du million suivant (j'ai finalement réussi à faire marcher tous les algorithmes dans un temps raisonnable).

Ce dataset contient de plus une indication sur la difficulté des grilles considérées. Cette difficulté est calculée à partir du nombre d'itérations nécessaires en moyenne pour un algorithme de référence à compléter ces grilles. Voir plus d'information [ici](https://www.kaggle.com/radcliffe/3-million-sudoku-puzzles-with-ratings).

Voici une idée de la difficulté et du nombre d'indications données pour ces grilles.

In [ ]:
import os
import sys
path = os.getcwd() 
sys.path.append(os.path.abspath(os.path.join(path, os.pardir)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

data = pd.read_csv("assets/data.csv")

_data_X = data["puzzle"].apply(lambda x : [int(i) if i != '.' else 0 for i in x ])
data_X = np.stack(_data_X.to_numpy()).reshape((700, 9, 9))

_data_Y = data["solution"].apply(lambda x : [int(i) for i in x])
data_Y = np.stack(_data_Y.to_numpy()).reshape((700, 9, 9))

data.head(3)

In [ ]:
def pretty_print(pb, solution):
    print('Problem                            Solution')
    print('+-------+-------+-------+          +-------+-------+-------+')
    for i in range(9):
        row1 = pb[i]
        row2 = solution[i]
        to_print = "| "
        for j in range(9):
            to_print += f"{row1[j] if row1[j] else '.'} "
            if j % 3 == 2:
                to_print += f"| "
        to_print += f"         | "
        for j in range(9):
            to_print += f"{row2[j]if row2[j] else '.'} "
            if j % 3 == 2:
                to_print += f"| "
        print(to_print)
        if i % 3 == 2:
            print('+-------+-------+-------+          +-------+-------+-------+')
        
pretty_print(data_X[0], data_Y[0])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5), sharey='row')
fig.suptitle("Description de la difficulté et du nombre d'indications données")
ax1.hist(data['difficulty'])
ax1.set_title('Difficulté')
ax2.hist(data['clues'], bins=range(21, 28))
ax2.set_title("Nombre d'indications");

Il est à noter que la difficulté n'est pas fonction du nombre d'indications données. En effet, certaines gilles peuvent par exemple avoir relativement peu d'indications mais en impliquer directement un grand nombre.

Ci-dessous, je télécharge simplement les données issues de mes expériences:
- df_iterations contient le nombre d'itérations nécessaires à résoudre chacun des problèmes pour les algorithmes considérés.
- df_solved contient le nombre de cases vides à la fin des algorithmes. Un nombre de cases vides différent de 0 indique que l'algorithme n'a pas trouvé la solution du problème.

In [ ]:
df_iterations = pd.read_csv('assets/df_it.csv')
df_solved = pd.read_csv('assets/df_so.csv')

df_iterations['difficulty'] = df_iterations['difficulty'].apply(round)
df_solved['difficulty'] = df_solved['difficulty'].apply(round)

df_solved_bool = df_solved[df_solved['difficulty'] < 5].copy()
df_solved_bool[["Backtrack", "MCTS", "DeepIterativeSolver", "AlphaSudoku"]] = \
    (df_solved_bool[["Backtrack", "MCTS", "DeepIterativeSolver", "AlphaSudoku"]] > 0 ) * 1

In [ ]:
df_iterations.head(3)

In [ ]:
df_solved.head(3)

<a id='partie_1'></a>

## Partie I: présentation de l'algorithme Backtrack

L'algorithme Backtrack est l'algorithme le plus simple et le plus efficace afin de résoudre une grille de Sudoku. En effet, celui-ci repose sur l'exploration de coups possibles avec la possibilité de revenir en arrière si on arrive à une impasse.

Le bon fonctionnement de cet algorithme repose sur une heuristique le rendant très efficace: lorsque l'on doit choisir une action, il vaut mieux choisir tout d'abord la contrainte à respecter (un chiffre différent sur chaque cellule d'une ligne, d'une colonne ou d'un carré, ou encore le fait que chaque chiffre doit apparaître au moins une fois dans ces ensembles) pouvant être satisfaite par le nombre minimum d'action, puis prendre une action au hasard dans cet ensemble.

Dans les faits, c'est ce que tout le monde fait intuitivement. Si une cellule n'a qu'un seul choix, le chiffre 5 par exemple, on mettra le chiffre 5 dans cette celulle avant de faire quoique ce soit d'autre. De même, si toutes les cellules présentes trois choix ou plus et qu'une cellule peut être remplie que par deux chiffres possiblement, on aura tendance à remplir cette dernière en premier. De même, si le chiffre 8 ne peut apparaître qu'une seule fois sur une ligne donnée, on remplira la cellule correspondante avec le chiffre 8 avant de passer à autre chose.

#### Résultats Backtrack

Vous pouvez voir ci-dessous les performances de cet algorithme sur les grilles testées:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Description des performances de l'algorithme BackTrack")
axes[0].boxplot(df_iterations["Backtrack"])
axes[0].set_title("Nombre d'itérations effectuées")
axes[0].set_ylim((0, 10000))
df_iterations[df_iterations['difficulty'] < 5].groupby("difficulty")['Backtrack'].apply(np.mean).plot.bar(rot=0)
axes[1].set_title("Moyenne itérations effectuées par niveau de difficulté")
plt.sca(axes[0])
plt.xticks([1], ['Backtrack Algorithm'])
plt.show()

<a id='partie_2'></a>

## Partie II: Approche Deep Learning

L'idée de cette partie est d'entraîner un réseau de neurones convolutionnel profond à prédire la solution d'une grille de Sudoku à partir du problème initial:
- les données en entrée et en sortie sont au format (9, 9, 9) : en effet, la première dimension est le nombre de channels de la grille initiale, avec un channel par valeur. Le channel i sera constitué de 1 là où il y a la valeur i + 1 sur la grille et de 0 partout ailleurs. Vous pouvez voir ci-dessous l'encoder construit pour transformer les grilles (9, 9) en ce nouveau format.
- la loss utilisée est une cross-entropy généralisée, car le format précédent nous permet de prédire les probabilités associées aux 9 classes possibles pour chaque cellule donnée.
- j'ai dû redéfinir une softmax afin de pouvoir respecter le format de mes données.
- le réseau est constitué de 8 couches de convolutions avec des noyaux au format (9, 1), (1, 9), (3, 3) et (9, 9) pour la première couche (le noyau (9, 9) est tout de même utile car je prends un padding 'same' afin de garder la dimension ; empiriquement, ajouter ce noyau améliorait les résultats) ; des noyaux plus classiques sont utilisés ensuite, avec une skip connection ajoutée sur l'avant-dernière couche, la reliant avec les entrées (afin de faciliter la tâche pour que le réseau prédise bien les bonnes valeurs lorsqu'elles sont données en entrée dans le problème initial). Le réseau ainsi obtenu a 1 million de paramètres.

Ce réseau sera également utilisé dans la partie IV.

Quant à l'algorithme utilisant uniquement cette approche Deep Learning, il est proposé à la suite de la présentation du réseau. Il consiste simplement à prédire les cellules une par une au lieu de les prédire toutes d'un coup.

Note: j'aurais pu obtenir un réseau plus performant en tirant partie de la symétrie du jeu, et en l'entraînant sur des grilles avec des nombres d'indices plus variés (allant de 20 à 70 par exemple, alors qu'ici je garde les grilles entre 20 et 26 environ). J'ai préféré me concentrer sur l'implémentation des algorithmes en eux-mêmes.

Voici le réseau en question. Il contient 1 million de paramètres et s'entraîne en 20 minutes sur les 2 millions de grilles évoquées ci-dessus (sur Colab avec un "boost" GPU). Le dataset d'entraînement au format (2000000, 9, 9, 9) rentre très bien dans la RAM à condition de préciser que tous les nombres sont des booléens et non des float.

<img src="assets/model.png" />

Les performances du réseau étaient d'environ $0.6$ pour la cross-entropie généralisée sur le dataset de test (3ème million de grilles), ce qui correspondait à environ $70 \%$ des cellules étant prédites de manière correcte. En revanche, presque aucune des grilles de test n'était prédite de manière entièrement correcte directement par le réseau. Il fallait donc tricher un peu et faire remplir les cellules une par une ! De plus, je guide le réseau en lui faisant remplir de manière non probabiliste les cellules ayant une unique possibilité.

Cet algorithme à un nombre d'itérations quasi-constant pour un temps d'exécution d'une seconde environ, car il ne revient jamais en arrière. En revanche, comme vous pouvez le voir ci-dessous, il échoue souvent ($40 \%$ des cas) à trouver la bonne solution.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Analyse des performances de DeepIterativeSolver")
axes[0].bar(x=['Succès', 'Echec'],
        height=[100 * (1 - df_solved_bool['DeepIterativeSolver'].mean()),
                100 * df_solved_bool['DeepIterativeSolver'].mean()],
       width=0.4)
axes[0].set_title('Succès vs échecs de DeepIterativeSolver')
df_solved_bool.groupby("difficulty")['DeepIterativeSolver'].apply(np.mean).plot.bar(rot=0);
axes[1].set_title("Taux d'échec en fonction de la difficulté")
plt.sca(axes[0])
plt.show()

Comme vous pouvez le voir, cet algorithme est également sensible à la difficulté des grilles ! Cela s'explique par le guidage lorsqu'une unique possibilité se présente. Comme nous pouvons le voir ci-dessous, le taux de succès semble moins lié au nombre d'indices donnés (même si c'est égalenement fortement corrélé) qu'à la difficulté des grilles.

In [ ]:
df_solved_bool[(df_solved_bool["clues"] < 28) &
                (df_solved_bool["clues"] > 21)].groupby("clues")['DeepIterativeSolver']\
    .apply(np.mean).plot.bar(title="Taux d'échec par indices donnés", rot=0);

<a id='partie_3'></a>

## Partie III: Monte Carlo Tree Search

Dans cette partie, j'implémente un algorithme de Monte Carlo Tree Search appliqué aux Sudokus.

Le principe de cet algorithme est d'explorer différents chemins à partir de l'état auquel on se trouve, en construisant un arbre d'enfants (noeuds) possibles avec pour chacun une valeur associée. Cet arbre est étendu un nombre arbitraire de fois, et les valeurs des noeuds sont calculées en allant jusqu'à un état terminal (en général en-dehors de l'arbre, par _rollout_ ; la _rollout policy_ utilisée ici est basée sur un choix aléatoire respectant l'heuristique donnée dans la partie I), et en faisant remonter la valeur obtenue avec un éventuel facteur de discount (que je fixe à 1 ici). A chaque embranchement de l'arbre, une nouvelle grille est choisie en balançant entre l'exploration et l'exploitation, selon une fonction de choix appelée _Upper Confidence Bounds for Trees_, définie comme la _tree policy_. Le poids théorique d'exploration est de $\sqrt{2}$, et comme cet algorithme est assez lent, je n'ai pas cherché à optimiser ce poids.

La fonction de "reward" utilisée ici est très simple et correspond simplement au nombre de cellules remplies dans les grilles terminales:
$$r(s, a) = \frac{nb cellules remplies}{81}$$
Je garde un facteur de discount à 1 puisque chaque action est également importante. Empiriquement, ce choix très simple de fonction de reward semble donner de bons résultats, même si un grand nombre de chemins est parfois exploré.

De même, je n'ai pas cherché à optimiser le nombre d'expansions de l'arbre considérées à chaque étape avant de choisir une action. Pour mes simulations, je l'ai fixé à 20, mais il faudrait probablement le fixer à 100 ou plus pour obtenir de meilleurs résultats.

Comme j'ai déjà pu le dire auparavant, à chaque embranchement, je ne considère que les cellules avec un nombre minimal de possibilités.

Note: Je me suis inspiré de [cette implémentation générale](#https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1) que j'ai adaptée aux sudokus. 

Comme vous pouvez le voir ci-dessous, le nombre d'itérations peut parfois exploser. De même que précédemment, le niveau de difficulté semble influencer de manière directe les performances de l'algorithme.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Description des performances de l'algorithme MCTS")
axes[0].boxplot(df_iterations["MCTS"])
axes[0].set_title("Nombre d'itérations effectuées")
df_iterations[df_iterations['difficulty'] < 5].groupby("difficulty")['MCTS'].apply(np.mean).plot.bar(rot=0)
axes[1].set_title("Moyenne itérations effectuées par niveau de difficulté")
plt.sca(axes[0])
plt.xticks([1], ['MCTS Algorithm'])
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Analyse des performances de MCTS")
axes[0].bar(x=['Succès', 'Echec'],
        height=[100 * (1 - df_solved_bool['MCTS'].mean()),
                100 * df_solved_bool['MCTS'].mean()],
       width=0.4)
axes[0].set_title('Succès vs échecs de MCTS')
df_solved_bool.groupby("difficulty")['MCTS'].apply(np.mean).plot.bar(rot=0);
axes[1].set_title("Taux d'échec en fonction de la difficulté")
plt.show()

#### Conclusion des partie II et III:
Parmi les deux algorithmes vus précédemment, l'un a un taux d'échec très élevé (DeepIterativeSolver avec $40 \%$), l'autre a un taux d'échec plus faible (MCTS avec $20 \%$), mais au prix d'un nombre d'itérations beaucoup plus élevé pour MCTS que pour DeepIterativeSolver, qui a un nombre d'itérations constant et très faible.

Aucun de ces algorithmes n'est satisfaisant, et nous allons essayer dans la partie suivante de créer un algorithme tirant parti des deux méthodes précédentes.

<a id='partie_4'></a>

## Partie IV: Alpha Sudoku

Dans cette partie, je reprends les idées de la partie II et de la partie III, et les utilise dans un cadre unifié similaire à AlphaGo [[5]](#bibliography).

En effet, la structure d'AlphaSudoku est très similaire à MCTS. De plus, je réutilise le réseau entraîné dans la partie II comme _rollout policy_ (je choisis les actions en me basant uniquement sur ce réseau), et utilise les probabilités calculées à partir de ce réseau pour la _tree policy_. En effet, pour cette dernière, et comme dans le cas d'AlphaGo je choisis l'action maximisant:

$$argmax \left(Q(s, a) + w * \frac{P(s, a)}{1 + N(s, a))}\right)$$

où $w$ est le poids accordé à l'exploration, $Q(s, a)$ est la valeur associée à l'action $a$ (à la grille $a$) à partir de l'état $s$ (la grille $s$), $P(s, a)$ sa probabilité calculée par le réseau et $N(s, a)$ le nombre de visites de cette action (grille) à partir de l'état considéré.

De même que dans la partie III, j'utilise simplement le nombre de cellules non-vides pour évaluer les grilles finales, avec un facteur de discount égal à 1.

Les différences par rapport à AlphaSudoku sont les suivantes:
- Je ne réentraîne pas le réseau avec des "parties" jouées en utilisant ce réseau.
- Je n'ai pas de value network, car j'utilise la fonction de reward très simple décrite dans la partie précédente.
- J'utilise le réseau à la fois comme _SL policy_ (utilisé dans AlphaGo pour choisir les mouvements pour étendre l'arbre) et comme _rollout policy_. Cela ralentit l'algorithme (Alpha Go choisi un réseau beaucoup plus simple pour la rollout policy afin d'aller plus vite) mais je peux me le permettre étant donné la simplicité du Sudoku par rapport au jeu de Go.
- Je n'ai pas tuné les hyperparamètres comme le nombre d'extensions des arbres considérés, que je fixe à 20, ou encore le poids d'exploration, que je fixe à $\sqrt{2}$.

Comme vous pouvez le voir sur les figures suivantes, le taux de succès d'AlphaSudoku semble très encourageant, avec $3 \%$ d'échecs. 

La difficulté semble influencer sur le taux d'échec, car sur les grilles les plus simples, cet algorithme semble se tromper dans moins de $2 \%$ des cas en moyenne, contre $7 \%$ environ sur les grilles les plus complexes.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Analyse des performances d'AlphaSudoku")
axes[0].bar(x=['Succès', 'Echec'],
        height=[100 * (1 - df_solved_bool['AlphaSudoku'].mean()),
                100 * df_solved_bool['AlphaSudoku'].mean()],
       width=0.4)
axes[0].set_title("Succès vs échecs d'AlphaSudoku")
df_solved_bool.groupby("difficulty")['AlphaSudoku'].apply(np.mean).plot.bar(rot=0);
axes[1].set_title("Taux d'échec en fonction de la difficulté")
plt.sca(axes[0])
plt.show()

Mais qu'en est-il des performances en termes d'itérations ?

Comme nous pouvons le voir sur les figures suivantes, elles semblent encore très raisonnables ! En effet, la grande majorité des problèmes requièrent moins de 500 itérations, avec des maxima vers 3000.

Mais passons à la partie suivante pour une meilleure comparaison des algorithmes vus jusqu'à maintenant.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Description des performances de l'algorithme AlphaSudoku")
axes[0].boxplot(df_iterations["AlphaSudoku"])
axes[0].set_title("Nombre d'itérations effectuées")
df_iterations[df_iterations['difficulty'] < 5].groupby("difficulty")['AlphaSudoku'].apply(np.mean).plot.bar(rot=0)
axes[1].set_title("Moyenne itérations effectuées par niveau de difficulté")
plt.sca(axes[0])
plt.xticks([1], ['AlphaSudoku Algorithm'])
plt.show()

<a id='partie_5'></a>

## Partie V: Comparaison, résultats

#### Nombre d'itérations

Commençons tout d'abord par nous intéresser au nombre d'itérations de nos algorithmes.

Comme nous pouvons le voir ici, DeepIterativeSolver est beaucoup plus rapide (mais au prix de plus d'échecs comme nous le verrons juste après). Vient ensuite AlphaSudoku, loin devant Backtrack et MCTS.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Analyse des performances")
axes[0].boxplot(df_iterations[['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku']])
axes[0].set_title("Nombre d'itérations")
axes[1].boxplot(df_iterations[['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku']])
axes[1].set_title("Nombre d'itérations (zoom)")
axes[1].set_ylim((0, 3000))
plt.setp(axes, xticks=[1, 2, 3, 4], xticklabels=['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku'])
plt.show()

Concernant le nombre d'itérations par niveau de difficulté, nous pouvons voir que **AlphaSudoku** et 
**MCTS** semblent moins sensibles au niveau de difficulté des grilles, et ont ainsi une durée relativement constante d'exécution sur les différents types de problème (en moyenne bien entendu).

In [ ]:
df_iterations[df_iterations['difficulty'] < 5].groupby("difficulty")\
    [['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku']].apply(np.mean).plot.bar(rot=0, title="Nombre moyen d'itérations par niveau de difficulté");

#### Taux d'échec des algorithmes

De même que précédemment, comparons les algorithmes globalement puis plus en détail en fonction de la difficulté des problèmes considérés. Encore une fois, la comparaison joue clairement en faveur d'AlphaSudoku pour les méthodes probabilistes.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle("Description des performances de l'algorithme AlphaSudoku")
axes[0].bar(['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku'], 
            df_solved_bool[['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku']].mean())
axes[0].set_title("Taux d'échec global")
df_diff_solved = df_solved_bool.groupby('difficulty')[['Backtrack', 'DeepIterativeSolver', 'MCTS', 'AlphaSudoku']].mean()
df_diff_solved.plot.bar(ax=axes[1], rot=0)
axes[1].set_title("Taux d'échec par niveau de difficulté")
plt.show()

## Conclusion

Le jeu de Sudoku se résout très bien par l'algorithme de Backtrack, qui fournit dans un temps raisonnable une solution de manière certaine. Cependant, nous avons pu voir dans ce projet que des méthodes probabilistes peuvent donner des résultats également très bons en termes d'itérations, au prix d'une perte en termes de succès. Dans le cas d'AlphaSudoku, cette perte est faible, et pourrait sans doute être améliorée par une optimisation assez simple.

En effet, comme j'ai déjà pu le mentionner, le fonction de reward est extrêmement simple et il est vraisemblable qu'elle puisse être améliorée (par exemple : 2 * nbcellulesnonvides / 81 - 1 afin de pénaliser les grilles s'arrêtant trop tôt, mais beaucoup d'autres possibilités pourraient être envisagées). De plus, je n'ai pas essayé d'optimiser les autres paramètres des modèles comme le poids donné à l'exploration par exemple. Enfin, je pourrais tirer parti de la symétrie du problème en faisant prédire chaque grille 4 fois, en faisant une rotation à chaque fois et en faisant voter les résultats obtenus (cela ralentirait l'algorithme mais pourrait donner de meilleurs résultats).

Il pourrait aussi être intéressant de comparer ces algorithmes en termes de temps de calcul. Si j'avais eu plus de temps, j'aurai cherché à optimiser le plus possible les algorithmes afin de pouvoir les comparer en ces termes. Mais même ainsi, comme j'ai pu le dire en introduction, les performances seraient dépendantes des machines plus que des algorithmes en eux-mêmes, ce qui m'a fait opter pour une comparaison en termes d'itérations.

Enfin, j'aurais aimé comparer ces algorithmes sur des grilles 16\*16 et non plus seulement 9\*9. Cela serait sans doute très intéressant car plus la dimension augmente, plus l'algorithme BackTrack prendra du temps à s'exécuter, alors qu'un algorithme comme AlphaSudoku semble moins impacté par la difficulté des problèmes. En revanche, l'entraînement serait alors un véritable challenge car il est peu probable que des millions de grilles 16\*16 soient disponibles aussi facilement que celles de taille 9\*9.

Ainsi, pour résoudre ce dernier problème, il serait intéressant de développer un algorithme apprenant seul à résoudre des grilles vides par exemple. Sans doute que cette approche serait plus efficace que l'algorithme Backtrack afin de créer de nouvelles grilles 16\*16! AlphaSudoku Zéro a de l'avenir ...

<a id='bibliography'></a>

## Bibliographie

[1] https://fr.wikipedia.org/wiki/Sudoku

[2] Jean-Paul Delahaye; <ins>_The Science behind Sudoku_</ins>, Scientific American, Vol. 294, No. 6, pp. 80–
87, 2006. ([paper](http://www.cs.virginia.edu/~robins/The_Science_Behind_SudoKu.pdf))

[3] Gary McGuire, Bastian Tugemann, Gilles Civario ; <ins>_There is no 16-Clue Sudoku: Solving the Sudoku Minimum Number of Clues Problem_</ins> January 2012, Experimental Mathematics 23. ([paper](https://arxiv.org/abs/1201.0749))

[4] Donald Knuth ; <ins>_Dancing Links_</ins> Perspectives in Computer Science, 2000, 187--214 ; ([paper](https://arxiv.org/abs/cs/0011047))

[5] Silver, D., Huang, A., Maddison, C. et al. <ins>_Mastering the game of Go with Deep Neural Networks & Tree Search_ </ins>  Nature 529, 484–489 (2016). ([paper](https://storage.googleapis.com/deepmind-media/alphago/AlphaGoNaturePaper.pdf))